In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configuración de estilo
plt.style.use('default')
sns.set_palette("husl", 7)
sns.set_context("talk")

In [9]:
def analyze_splus_internal_consistency(df, aperture=4):
    """
    Analizar la consistencia interna de la fotometría SPLUS
    """
    # Primero, asegurémonos de que todas las columnas numéricas sean realmente numéricas
    def safe_numeric_conversion(series):
        """Convertir una serie a numérico, manejando valores no numéricos"""
        return pd.to_numeric(series, errors='coerce')
    
    # Convertir todas las columnas de magnitud y error a numéricas
    for col in df.columns:
        if col.startswith(('MAG_', 'MAGERR_', 'FLUX_', 'FLUXERR_')):
            df[col] = safe_numeric_conversion(df[col])
    
    # Definir los filtros SPLUS en orden de longitud de onda
    filters = ['F378', 'F395', 'F410', 'F430', 'F515', 'F660', 'F861']
    filter_names = ['F378 (u)', 'F395 (u)', 'F410 (CaHK)', 'F430 (G-band)', 
                   'F515 (g)', 'F660 (r)', 'F861 (CaT)']
    
    # Crear máscara de datos válidos
    valid_mask = np.ones(len(df), dtype=bool)
    for filt in filters:
        mag_col = f'MAG_{filt}_{aperture}'
        if mag_col in df.columns:
            # Filtrar valores no válidos (99.0, NaN, y fuera de rango razonable)
            valid_mask &= (df[mag_col] < 90) & (df[mag_col] > 10) & (~df[mag_col].isna())
    
    df_valid = df[valid_mask].copy()
    
    print(f"Fuentes con fotometría completa: {len(df_valid)}")
    
    if len(df_valid) == 0:
        print("No hay suficientes datos válidos para el análisis")
        return df
    
    # 1. Distribución de magnitudes por filtro
    plt.figure(figsize=(14, 8))
    for i, filt in enumerate(filters):
        mag_col = f'MAG_{filt}_{aperture}'
        if mag_col in df_valid.columns:
            # Asegurarnos de que los datos sean numéricos
            mag_data = pd.to_numeric(df_valid[mag_col], errors='coerce')
            mag_data = mag_data[~mag_data.isna() & (mag_data < 90) & (mag_data > 10)]
            if len(mag_data) > 0:
                plt.hist(mag_data, bins=30, alpha=0.7, label=filter_names[i])
    
    plt.xlabel('Magnitud')
    plt.ylabel('Número de cúmulos')
    plt.title('Distribución de magnitudes por filtro (apertura de 4 arcsec)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('splus_magnitude_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Diagramas de color-color
    # Calcular todos los colores primero
    colors = {}
    for i in range(len(filters)-1):
        for j in range(i+1, len(filters)):
            color_name = f'{filters[i]}-{filters[j]}'
            mag1_col = f'MAG_{filters[i]}_{aperture}'
            mag2_col = f'MAG_{filters[j]}_{aperture}'
            
            if mag1_col in df_valid.columns and mag2_col in df_valid.columns:
                # Asegurar que ambas columnas sean numéricas
                mag1 = pd.to_numeric(df_valid[mag1_col], errors='coerce')
                mag2 = pd.to_numeric(df_valid[mag2_col], errors='coerce')
                
                # Filtrar valores válidos
                valid_mask = (~mag1.isna()) & (~mag2.isna()) & (mag1 < 90) & (mag2 < 90) & (mag1 > 10) & (mag2 > 10)
                colors[color_name] = mag1[valid_mask] - mag2[valid_mask]
    
    # Definir pares de colores para graficar
    color_pairs = [
        ('F515-F660', 'F660-F861'),  # (g-r) vs (r-i) - estándar
        ('F378-F395', 'F395-F410'),   # UV
        ('F410-F430', 'F430-F515'),   # Azul
        ('F515-F660', 'F378-F395'),   # (g-r) vs UV
        ('F660-F861', 'F395-F410'),   # (r-i) vs UV intermedio
    ]
    
    # Crear diagramas color-color
    n_plots = len(color_pairs)
    n_cols = 2
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 6*n_rows))
    axes = axes.ravel()
    
    for idx, (x_color, y_color) in enumerate(color_pairs):
        if x_color in colors and y_color in colors and len(colors[x_color]) > 0 and len(colors[y_color]) > 0:
            ax = axes[idx]
            
            # Usar solo los puntos donde ambos colores tienen valores
            valid_mask = (~pd.Series(colors[x_color]).isna()) & (~pd.Series(colors[y_color]).isna())
            x_vals = colors[x_color][valid_mask]
            y_vals = colors[y_color][valid_mask]
            
            if len(x_vals) > 0 and len(y_vals) > 0:
                # Calcular color de referencia para el colorbar
                if 'F515-F660' in colors:
                    color_vals = colors['F515-F660'][valid_mask]
                else:
                    color_vals = np.zeros(len(x_vals))
                
                sc = ax.scatter(x_vals, y_vals, alpha=0.7, s=30, c=color_vals, cmap='viridis')
                
                ax.set_xlabel(x_color)
                ax.set_ylabel(y_color)
                ax.set_title(f'{y_color} vs {x_color}')
                ax.grid(True, alpha=0.3)
                
                # Añadir barra de color para (g-r) si está disponible
                if 'F515-F660' in colors:
                    plt.colorbar(sc, ax=ax, label='Color F515-F660 (g-r)')
    
    # Ocultar ejes vacíos si los hay
    for idx in range(len(color_pairs), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig('splus_color_color_diagrams.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. Curvas de energía espectral (SEDs) para cúmulos típicos
    # Longitudes de onda efectivas de los filtros SPLUS (en Angstrom)
    wave_eff = {
        'F378': 3770, 'F395': 3940, 'F410': 4094, 
        'F430': 4292, 'F515': 5133, 'F660': 6614, 'F861': 8611
    }
    
    # Seleccionar algunos cúmulos representativos
    representative_gcs = []
    if 'Prob' in df_valid.columns:
        # Seleccionar cúmulos con alta probabilidad
        high_prob_gcs = df_valid[df_valid['Prob'] > 0.8].index[:5]
        representative_gcs.extend(high_prob_gcs)
    
    # Si no hay suficientes, seleccionar aleatoriamente
    if len(representative_gcs) < 5:
        additional = min(5, len(df_valid))
        additional_gcs = np.random.choice(df_valid.index, additional, replace=False)
        representative_gcs.extend(additional_gcs)
    
    # Graficar SEDs
    plt.figure(figsize=(12, 8))
    for gc_idx in representative_gcs[:5]:  # Mostrar solo 5 para claridad
        if gc_idx in df_valid.index:
            gc_data = df_valid.loc[gc_idx]
            fluxes = []
            wavelengths = []
            
            for filt in filters:
                mag_col = f'MAG_{filt}_{aperture}'
                if mag_col in gc_data and pd.notna(gc_data[mag_col]) and gc_data[mag_col] < 90:
                    # Convertir magnitud a flujo (arbitrario)
                    flux = 10**(-0.4 * gc_data[mag_col])
                    fluxes.append(flux)
                    wavelengths.append(wave_eff[filt])
            
            if len(fluxes) > 3:  # Solo graficar si hay suficientes puntos
                # Normalizar flujos para mejor visualización
                fluxes = np.array(fluxes)
                fluxes /= np.max(fluxes)
                plt.plot(wavelengths, fluxes, 'o-', label=f'GC {gc_idx}')
    
    plt.xlabel('Longitud de onda (Å)')
    plt.ylabel('Flujo normalizado')
    plt.title('Curvas de energía espectral para cúmulos globulares representativos')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('splus_gc_seds.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 4. Análisis de errores fotométricos (con verificación de tipos de datos)
    plt.figure(figsize=(14, 8))
    for i, filt in enumerate(filters):
        mag_col = f'MAG_{filt}_{aperture}'
        err_col = f'MAGERR_{filt}_{aperture}'
        
        if mag_col in df_valid.columns and err_col in df_valid.columns:
            # Asegurarnos de que los datos sean numéricos
            mag_data = pd.to_numeric(df_valid[mag_col], errors='coerce')
            err_data = pd.to_numeric(df_valid[err_col], errors='coerce')
            
            # Filtrar valores válidos
            valid_mask = (~mag_data.isna()) & (~err_data.isna()) & (mag_data < 90) & (mag_data > 10) & (err_data < 10) & (err_data > 0)
            
            if np.any(valid_mask):
                plt.scatter(mag_data[valid_mask], err_data[valid_mask], 
                           alpha=0.6, label=filter_names[i], s=20)
    
    plt.xlabel('Magnitud')
    plt.ylabel('Error de magnitud')
    plt.title('Relación magnitud-error para los filtros SPLUS')
    plt.legend()
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.savefig('splus_mag_error_relation.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 5. Matriz de correlación entre colores
    # Crear DataFrame con todos los colores
    color_df = pd.DataFrame()
    for color_name, color_values in colors.items():
        if len(color_values) > 0:
            color_df[color_name] = color_values
    
    if not color_df.empty and len(color_df) > 1:
        # Calcular matriz de correlación
        corr_matrix = color_df.corr()
        
        # Graficar matriz de correlación
        plt.figure(figsize=(12, 10))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', 
                   center=0, square=True, fmt='.2f')
        plt.title('Matriz de correlación entre colores SPLUS')
        plt.tight_layout()
        plt.savefig('../anac_data/splus_color_correlation_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # 6. Estadísticas resumen
    print("\n" + "="*60)
    print("ESTADÍSTICAS DE CONSISTENCIA INTERNA SPLUS")
    print("="*60)
    
    stats_summary = []
    for filt in filters:
        mag_col = f'MAG_{filt}_{aperture}'
        if mag_col in df_valid.columns:
            mag_data = pd.to_numeric(df_valid[mag_col], errors='coerce')
            mag_data = mag_data[~mag_data.isna() & (mag_data < 90) & (mag_data > 10)]
            
            if len(mag_data) > 0:
                stats_summary.append({
                    'Filtro': filt,
                    'N': len(mag_data),
                    'Mediana': np.median(mag_data),
                    'Media': np.mean(mag_data),
                    'Std': np.std(mag_data),
                    'MAD': np.median(np.abs(mag_data - np.median(mag_data))),
                    'Min': np.min(mag_data),
                    'Max': np.max(mag_data)
                })
    
    if stats_summary:
        stats_df = pd.DataFrame(stats_summary)
        print(stats_df.to_string(index=False))
    else:
        print("No hay suficientes datos para generar estadísticas")
    
    return df_valid

In [10]:
# Cargar datos
df = pd.read_csv('../anac_data/all_fields_gc_photometry_merged.csv')


In [11]:
# Ejecutar análisis de consistencia interna
df_valid = analyze_splus_internal_consistency(df, aperture=4)

Fuentes con fotometría completa: 95

ESTADÍSTICAS DE CONSISTENCIA INTERNA SPLUS
Filtro  N   Mediana     Media      Std      MAD       Min       Max
  F378 95 21.268695 21.374932 0.881946 0.505951 18.912572 23.893796
  F395 95 21.050277 21.226144 1.062197 0.485977 18.804412 25.735792
  F410 95 20.771919 20.839359 1.001166 0.533195 18.398224 25.813244
  F430 95 20.720694 20.905308 1.240611 0.560188 18.397141 26.920607
  F515 95 20.165925 20.102122 0.836321 0.575661 17.858403 22.113127
  F660 95 19.572995 19.499857 0.785141 0.564400 17.529885 21.040064
  F861 95 19.200354 19.117137 0.769162 0.543763 17.326377 20.761475
